In [1]:
#!pip install "runhouse[aws]" torch datasets transformers peft trl
import runhouse as rh 
import os

#os.chdir("/dir/mydir")
from LoraFineTuner import FineTuner

INFO | 2024-08-09 19:49:00.718467 | Loaded Runhouse config from /Users/paulyang/.rh/config.yaml
/opt/anaconda3/envs/tempenv/lib/python3.11/site-packages/asyncssh/crypto/cipher.py:29: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/opt/anaconda3/envs/tempenv/lib/python3.11/site-packages/asyncssh/crypto/cipher.py:30: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import TripleDES
/opt/anaconda3/envs/tempenv/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": alg

## Connect to cluster

In [4]:
# Reqs will be installed by Runhouse on remote
# We can also show you how to launch with a Docker container / conda env 
img = rh.Image(name="ft_env").install_packages([
        "torch",
        "tensorboard",
        "scipy",
        "peft==0.4.0",
        "bitsandbytes==0.40.2",
        "transformers==4.31.0",
        "trl==0.4.7",
        "accelerate",
    ])

cluster = rh.cluster(
    name="rh-a10x",
    instance_type="A10G:1",
    memory="32+",
    provider="aws",
    image = img 
).up_if_not()

# You will need a HF_TOKEN as an env variable to download the pretrained model in this example
cluster.sync_secrets(["huggingface"])


## Send fine tuner to remote and instantiate / Get already instantiated remote instance
* There is a locally defined LoraFineTuner class
* Runhouse will send this class to remote compute
* Then locally we create an *instance* of this remote class, which we name `rh_finetuner` or anything else
* We call this remote-instance from local as if it were normal/local, and can access it by name from any Python session connected to the cluster

In [7]:
fine_tuner_remote_name = "rh_finetuner" ## This is the name of the *instance* of the remote class, not the remote class

# We check if we have already created a "rh_finetuner" on the remote which is an *instance* of the remote fine tuner class
fine_tuner_remote = cluster.get(fine_tuner_remote_name, default=None, remote=True)

# If we have not, then we will send the local class to remote, and create an instance of it named "rh_finetuner"
# If you disconnect locally after calling tune, you can simply reconnect to the remote object using this block from another local session
if fine_tuner_remote is None:
    fine_tuner = rh.module(FineTuner).to(
        cluster, env=env, name="llama3-medical-model"
    )
    fine_tuner_remote = fine_tuner(name=fine_tuner_remote_name)

In [8]:
## Once we have accessed the remote class, we can call against it as if it were a local object 
fine_tuner_remote.tune()

INFO | 2024-08-09 20:52:33.269630 | Calling rh_finetuner.tune



Map:  59%|█████▉    | 1000/1700 [00:00<00:00, 5934.16 examples/s]ft_env env: Calling method tune on module rh_finetuner
ft_env env: Calling method training_params on module rh_finetuner
ft_env env: Calling method sft_trainer on module rh_finetuner

Map: 100%|██████████| 1700/1700 [00:00<00:00, 5551.21 examples/s]

Map: 100%|██████████| 300/300 [00:00<00:00, 5197.45 examples/s]

  6%|▌         | 25/425 [00:57<07:43,  1.16s/it]
                                                

  6%|▌         | 25/425 [00:57<07:43,  1.16s/it]{'loss': 1.9178, 'learning_rate': 0.0002, 'epoch': 0.06}

 12%|█▏        | 50/425 [01:15<03:27,  1.81it/s]
                                                

 12%|█▏        | 50/425 [01:15<03:27,  1.81it/s]{'loss': 1.4426, 'learning_rate': 0.0002, 'epoch': 0.12}

 18%|█▊        | 75/425 [02:08<06:08,  1.05s/it]
                                                

 18%|█▊        | 75/425 [02:08<06:08,  1.05s/it]{'loss': 1.5318, 'learning_rate': 0.0002, 'epoch': 0.18}

 24

INFO | 2024-08-09 21:02:36.506223 | Time to call rh_finetuner.tune: 603.24 seconds


In [13]:
# Once the fine tuner is complete, we can query against it 
query = "What's the best treatment for sunburn?"
generated_text = fine_tuner_remote.generate(query, max_length = 1000)
print(generated_text)


INFO | 2024-08-09 14:41:07.557128 | Calling rh_finetuner.generate


ft_env env: Calling method generate on module rh_finetuner
ft_env env: Calling method load_pipeline on module rh_finetuner


INFO | 2024-08-09 14:41:08.122288 | Time to call rh_finetuner.generate: 0.57 seconds


<|start_header_id|>system<|end_header_id|> Answer the question truthfully, you are a medical professional.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: What's the best treatment for sunburn?<|eot_id|><|start_header_id|>assistant<|end_header_id|> The best treatment for sunburn is to prevent it.


## I can reconnect this remote instance of the fine tuner even after my local session disconnects, or from another session
If I connect to the cluster, and get the object by name, I can call against it even if my local session ends

You can run the below code at any point, including while training is running, from anywhere (like another notebook)

In [9]:
import runhouse as rh 

# We check if we have already created a "rh_finetuner" on the remote which is an *instance* of the remote fine tuner class
cluster = rh.cluster(name="/rh-alpha-testers/jamesb")
fine_tuner_remote_name = "rh_finetuner"
fine_tuner_remote = cluster.get(fine_tuner_remote_name, default=None, remote=True)

# Check what the training status is on remote
if fine_tuner_remote is not None:
    print(fine_tuner_remote.get_training_status())

INFO | 2024-08-09 21:09:56.365787 | SSH tunnel on to server's port 32300 via server's ssh port 22 already created with the cluster.
INFO | 2024-08-09 21:09:57.109180 | Calling rh_finetuner.get_training_status


------
jamesb
------
ft_env env: Calling method get_training_status on module rh_finetuner
ft_env env: Calling method new_model_exists on module rh_finetuner


INFO | 2024-08-09 21:09:57.634861 | Time to call rh_finetuner.get_training_status: 0.53 seconds


{'tokenizer_loaded': True, 'model_loaded': True, 'pipeline_loaded': True, 'training_completed': True, 'epochs_trained': 2, 'eval results': None}
